In [2]:
!pip install pytorch-forecasting
import torch
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.tuner import Tuner

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.metrics import MultiHorizonMetric

from sklearn.preprocessing import LabelEncoder
from pytorch_forecasting.metrics import MultiHorizonMetric

import random
import os
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import pickle

Mounted at /content/drive


In [4]:
# Build Dataset에서 생성한 train_df와 test_df를 불러와야 합니다.
# sample_submission_csv는 데이콘에서 제공한 데이터입니다.

train_df_path = #경로지정#
test_df_path = #경로지정#
sample_submission_path = #경로지정#

train_df = pd.read_parquet(train_df_path)
test_df = pd.read_parquet(test_df_path)
sample_submission_csv = pd.read_csv(sample_submission_path)
train_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [ ]:
train_df.head()

,time,month,time_idx,week_weekend,day,special_day,product_nums,product,major,middle,...,product_info,product_info_label,sales_rate_log,date_cos,date_sin,day_cos,day_sin,month_cos,month_sin,weight
0,2022-01-01,01,0,1,5,1,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,...,헤어타입:모든 모발용 제품형태:스프레이형 주요제품특징:머릿결개선 주요제품특징:흡수력...,2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.310636
1,2022-01-01,01,0,1,5,1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,...,,4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.972058
2,2022-01-01,01,0,1,5,1,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,...,,3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.972058
3,2022-01-01,01,0,1,5,1,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,...,,4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.972058
4,2022-01-01,01,0,1,5,1,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,...,,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.762870


In [5]:
train_df['month'] = train_df['month'].astype(str)
train_df['time_idx'] = train_df['time_idx'].astype(int)
train_df['sales_rate'] = train_df['sales_rate'].astype(float)
train_df['week_weekend'] = train_df['week_weekend'].astype(str)
train_df['special_day'] = train_df['special_day'].astype(str)

max_prediction_length = 21
min_prediction_length = 1
max_encoder_length = 90

training = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target="sales_rate",
    group_ids=['product_nums'],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=min_prediction_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["product", "major", "middle", 'sub', 'brand'],
    static_reals=[],
    time_varying_known_categoricals=['month', 'week_weekend', 'special_day'],
    time_varying_known_reals=[],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["average_month_sales_rate",'sales_rate_log', 'sales_rate','sales',"keyword_cnt"],
    target_normalizer=GroupNormalizer(groups=["product_nums"], method="standard"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

In [6]:
validation = TimeSeriesDataSet.from_dataset(training, train_df, predict=True, stop_randomization=True)
batch_size = 1589  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8,  batch_sampler='synchronized')
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=8,  batch_sampler='synchronized')

In [7]:
class PSFA(MultiHorizonMetric):
    def loss(self, y_pred, target):
        y_pred = self.to_prediction(y_pred)
        diff_value = torch.abs(target - y_pred)
        max_value = torch.max(target, y_pred) + 1e-8
        weight = target / (torch.sum(target, axis=0).view(1,y_pred.shape[1]) + 1e-8)
        loss = ((diff_value / max_value) * weight) * (y_pred.shape[0])
        return loss

class SMAPE(MultiHorizonMetric):
    def loss(self, y_pred, target):
        y_pred = self.to_prediction(y_pred)
        loss = 2 * (y_pred - target).abs() / (y_pred.abs() + target.abs() + 1e-8)
        return loss

In [8]:
best_model_path = #훈련된 모델의 경로를 지정하면 됩니다#
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path).to(device)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


In [9]:
predictions = best_tft.predict(val_dataloader, trainer_kwargs=dict(accelerator="gpu"),  return_x=True, return_y = True)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [10]:
submission = sample_submission_csv.drop(columns=['ID'])
submission = pd.DataFrame(F.relu(torch.round(predictions.output)).detach().cpu().numpy()).astype(int)
submission = pd.concat([sample_submission_csv['ID'], submission], axis = 1)
submission.columns = sample_submission_csv.columns